In [23]:
from sklearn.feature_selection import mutual_info_regression
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report)
from sklearn.exceptions import NotFittedError
from sklearn.impute import SimpleImputer

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier,GradientBoostingClassifier, BaggingClassifier, HistGradientBoostingClassifier, StackingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import ADASYN

from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.base import FunctionSampler
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

In [2]:
data= pd.read_csv('/kaggle/input/framingham/framingham.csv')
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,male,39,4.0,No,0.0,0.0,0,0,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,female,46,2.0,No,0.0,0.0,0,0,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,male,48,1.0,Yes,20.0,0.0,0,0,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,female,61,3.0,Yes,30.0,0.0,0,1,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,female,46,3.0,Yes,23.0,0.0,0,0,No,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,female,48,2.0,Yes,20.0,NaN,0,0,No,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,female,44,1.0,Yes,15.0,0.0,0,0,No,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,female,52,2.0,No,0.0,0.0,0,0,No,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,male,40,3.0,No,0.0,0.0,0,1,No,185.0,141.0,98.0,25.60,67.0,72.0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sex              4240 non-null   object 
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   object 
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   object 
 9   totChol          4190 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4221 non-null   float64
 13  heartRate        4239 non-null   float64
 14  glucose          3852 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(4), object(3)
memory usage: 530.1+ KB


In [4]:
round(data.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
age,4240.0,49.58,8.57,32.00,42.00,49.0,56.00,70.0
education,4135.0,1.98,1.02,1.00,1.00,2.0,3.00,4.0
cigsPerDay,4211.0,9.01,11.92,0.00,0.00,0.0,20.00,70.0
BPMeds,4187.0,0.03,0.17,0.00,0.00,0.0,0.00,1.0
prevalentStroke,4240.0,0.01,0.08,0.00,0.00,0.0,0.00,1.0
prevalentHyp,4240.0,0.31,0.46,0.00,0.00,0.0,1.00,1.0
totChol,4190.0,236.70,44.59,107.00,206.00,234.0,263.00,696.0
sysBP,4240.0,132.35,22.03,83.50,117.00,128.0,144.00,295.0
diaBP,4240.0,82.90,11.91,48.00,75.00,82.0,90.00,142.5
BMI,4221.0,25.80,4.08,15.54,23.07,25.4,28.04,56.8


In [5]:
data.duplicated().sum()

0

In [6]:
# from ydata_profiling import ProfileReport
# profile=ProfileReport(data,explorative=True,dark_mode=True)
# profile

In [7]:
data.columns

Index(['Sex', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')

In [8]:
unique_values= {}
for i in data.columns:
    unique_values[i]= data[i].nunique()
    
pd.DataFrame(unique_values,index= ['unique_value']).T

,unique_value
Sex,2
age,39
education,4
currentSmoker,2
cigsPerDay,33
BPMeds,2
prevalentStroke,2
prevalentHyp,2
diabetes,2
totChol,248


In [9]:
data.isnull().sum()

Sex                  0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [10]:
data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})
data['currentSmoker']= data['currentSmoker'].map({'Yes':1, 'No':0})
data['diabetes']= data['diabetes'].map({'Yes':1, 'No':0})
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [11]:
target = data['TenYearCHD']
columns_to_drop = ['TenYearCHD']
features = data.drop(columns_to_drop, axis=1)

X = features.values
y = target.values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#xx_train, xx_test, yy_train, yy_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
#data2 = pd.concat([pd.DataFrame(xx_train), pd.DataFrame(yy_train)], axis=1)

# MICE:

In [13]:
import statsmodels.api as sm

imp = sm.MICEData(data)
data = imp.data
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,0.0,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,82.0,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


# Imputure

In [14]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# df= pd.DataFrame(data)
# imputer= IterativeImputer(max_iter=3, random_state=42)

# impute_data= imputer.fit_transform(df)
# data= pd.DataFrame(impute_data, columns= df.columns)

# data

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sex              4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   education        4240 non-null   float64
 3   currentSmoker    4240 non-null   int64  
 4   cigsPerDay       4240 non-null   float64
 5   BPMeds           4240 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   int64  
 9   totChol          4240 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4240 non-null   float64
 13  heartRate        4240 non-null   float64
 14  glucose          4240 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 530.1 KB


In [16]:
data.corr()['TenYearCHD'].sort_values(ascending= False)

TenYearCHD         1.000000
age                0.225408
sysBP              0.216374
prevalentHyp       0.177458
diaBP              0.145112
glucose            0.120442
diabetes           0.097344
Sex                0.088374
BPMeds             0.086448
totChol            0.081813
BMI                0.074787
prevalentStroke    0.061823
cigsPerDay         0.057647
heartRate          0.022897
currentSmoker      0.019448
education         -0.053570
Name: TenYearCHD, dtype: float64

In [17]:
# x_train, x_test,y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# classifiers = {
#     "RandomForest": RandomForestClassifier(random_state=42),
#     "AdaBoost": AdaBoostClassifier(random_state=42),
#     "ExtraTrees": ExtraTreesClassifier(n_estimators=1500, random_state=42),
#     "GradientBoosting": GradientBoostingClassifier(random_state=42),
#     "Bagging": BaggingClassifier(random_state=42),
#     "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
#     "DecisionTree": DecisionTreeClassifier(random_state=42),
#     "LogisticRegression": LogisticRegression(random_state=42),
#     "GaussianNB": GaussianNB(),
#     "BernoulliNB": BernoulliNB(),
#     "MultinomialNB": MultinomialNB(),
#     "KNeighbors": KNeighborsClassifier(),
#     #"RadiusNeighbors": RadiusNeighborsClassifier(),
#     "SVC": SVC(probability=True, random_state=42),
#     "XGBoost": XGBClassifier(random_state=42),
#     "LightGBM": LGBMClassifier(random_state=42),
#     "MLPClassifier": MLPClassifier(random_state=42)
# }
# results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'Specificity', 'ROC-AUC', 'Confusion Matrix'])

# def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
#     global results_df
#     for name, clf in classifiers.items():
#         try:
#             clf.fit(x_train, y_train) 
#             y_pred = clf.predict(x_test)  

#             cm = confusion_matrix(y_test, y_pred)
#             if len(cm.ravel()) == 4:  
#                 tn, fp, fn, tp = cm.ravel()
#                 specificity = tn / (tn + fp) 
#             else:
#                 specificity = None

#             acc = accuracy_score(y_test, y_pred)
#             f1 = f1_score(y_test, y_pred, average='weighted')
#             report = classification_report(y_test, y_pred, output_dict=True)
#             precision = report['weighted avg']['precision']
#             recall = report['weighted avg']['recall']

#             if hasattr(clf, "predict_proba"): 
#                 y_proba = clf.predict_proba(x_test)[:, 1]
#                 roc_auc = roc_auc_score(y_test, y_proba)
#             else:
#                 roc_auc = None
                
#             new_row = pd.DataFrame({
#                 'Model': [name],
#                 'Accuracy': [acc],
#                 'F1 Score': [f1],
#                 'Precision': [precision],
#                 'Recall': [recall],
#                 'Specificity': [specificity],
#                 'ROC-AUC': [roc_auc],
#                 'Confusion Matrix': [cm]
#             })
#             results_df = pd.concat([results_df, new_row], ignore_index=True)

#         except (ValueError, NotFittedError) as e:
#             print(f"Error with classifier {name}: {e}")
            
# evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
# print(results_df)

In [18]:
# def compute_knn(X, y, minority_class, k):
#     minority_instances = X[y == minority_class]
#     nbrs = NearestNeighbors(n_neighbors=k).fit(X)
#     distances, indices = nbrs.kneighbors(minority_instances)
#     return indices

# def identify_borderline_instances(X, y, indices, minority_class, k):
#     borderline_instances = []
#     for i, idx in enumerate(indices):
#         ksame = sum(y[idx] == minority_class)
#         kmajority = k - ksame
#         if ksame < k and kmajority > ksame:
#             borderline_instances.append(i)
#     return borderline_instances


# def generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples):
#     synthetic_samples = []
#     num_borderline = len(borderline_indices)
#     # প্রতি বর্ডারলাইন ইনস্ট্যান্স থেকে কিছু সিনথেটিক ডেটা তৈরি করবো
#     samples_per_instance = num_samples // num_borderline
#     for i in borderline_indices:
#         for _ in range(samples_per_instance):
#             rand_idx = np.random.choice(indices[i])
#             xj = X[rand_idx]
#             xi = X[i]
#             # Interpolation
#             rand = np.random.rand()
#             xnew = xi + rand * (xi - xj)
#             synthetic_samples.append(xnew)
#     # কিছু বাকি ডেটা যদি থেকে যায় (যদি ভাগফল পূর্ণ সংখ্যা না হয়)
#     remaining_samples = num_samples - len(synthetic_samples)
#     for _ in range(remaining_samples):
#         i = np.random.choice(borderline_indices)
#         rand_idx = np.random.choice(indices[i])
#         xj = X[rand_idx]
#         xi = X[i]
#         rand = np.random.rand()
#         xnew = xi + rand * (xi - xj)
#         synthetic_samples.append(xnew)
#     return np.array(synthetic_samples)


# def smote(X, y, minority_class, k, num_samples):
#     indices = compute_knn(X, y, minority_class, k)
#     borderline_indices = identify_borderline_instances(X, y, indices, minority_class, k)
#     synthetic_samples = generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples)
    
#     X_new = np.vstack((X, synthetic_samples))
#     y_new = np.hstack((y, np.array([minority_class] * len(synthetic_samples))))
    
#     return X_new, y_new

# if __name__ == "__main__":
#     target = data['TenYearCHD']
#     columns_to_drop = ['TenYearCHD']
#     features = data.drop(columns_to_drop, axis=1)

#     X = features.values
#     y = target.values

#     count_zeros_original = np.sum(y == 0)
#     count_ones_original = np.sum(y == 1)

#     print(f"Original Count of 0s (No Heart Attack): {count_zeros_original}")
#     print(f"Original Count of 1s (Heart Attack): {count_ones_original}")

#     minority_class = 1
#     k = 5
#     num_samples = 333

#     X_new, y_new = smote(X, y, minority_class, k, num_samples)

#     x_train, x_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

#     model = RandomForestClassifier()
#     model.fit(x_train, y_train)

#     y_pred = model.predict(x_test)
#     print(classification_report(y_test, y_pred))

#     count_zeros_new = np.sum(y_new == 0)
#     count_ones_new = np.sum(y_new == 1)

#     print(f"Count of 0s (No Heart Attack): {count_zeros_new}")
#     print(f"Count of 1s (Heart Attack): {count_ones_new}")

In [19]:
# def compute_knn(X, y, minority_class, k):
#     minority_instances = X[y == minority_class]
#     nbrs = NearestNeighbors(n_neighbors=k).fit(X)
#     distances, indices = nbrs.kneighbors(minority_instances)
#     return indices

# def identify_borderline_instances(X, y, indices, minority_class, k):
#     borderline_instances = []
#     for i, idx in enumerate(indices):
#         ksame = sum(y[idx] == minority_class)
#         kmajority = k - ksame
#         if ksame < k and kmajority > ksame:
#             borderline_instances.append(i)
#     return borderline_instances

# def generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples):
#     synthetic_samples = []
#     num_borderline = len(borderline_indices)

#     samples_per_instance = num_samples // num_borderline
#     for i in borderline_indices:
#         for _ in range(samples_per_instance):
#             rand_idx = np.random.choice(indices[i])
#             xj = X[rand_idx]
#             xi = X[i]
#             rand = np.random.rand()
#             xnew = xi + rand * (xj - xi)
#             synthetic_samples.append(xnew)

#     remaining_samples = num_samples - len(synthetic_samples)
#     for _ in range(remaining_samples):
#         i = np.random.choice(borderline_indices)
#         rand_idx = np.random.choice(indices[i])
#         xj = X[rand_idx]
#         xi = X[i]
#         rand = np.random.rand()
#         xnew = xi + rand * (xj - xi)
#         synthetic_samples.append(xnew)
#     return np.array(synthetic_samples)

# def smote(X, y, minority_class, k, num_samples):
#     indices = compute_knn(X, y, minority_class, k)
 
#     borderline_indices = identify_borderline_instances(X, y, indices, minority_class, k)
#     synthetic_samples = generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples)
    
#     X_new = np.vstack((X, synthetic_samples))
#     y_new = np.hstack((y, np.array([minority_class] * len(synthetic_samples))))
    
#     return X_new, y_new

# if __name__ == "__main__":
#     target = data['TenYearCHD']
#     columns_to_drop = ['TenYearCHD']
#     features = data.drop(columns_to_drop, axis=1)

#     X = features.values
#     y = target.values

#     count_zeros_original = np.sum(y == 0)
#     count_ones_original = np.sum(y == 1)

#     print(f"Original Count of 0s (No Heart Attack): {count_zeros_original}")
#     print(f"Original Count of 1s (Heart Attack): {count_ones_original}")

#     minority_class = 1
#     k = 5
#     num_samples = 333

#     X_new, y_new = smote(X, y, minority_class, k, num_samples)

#     x_train, x_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

#     model = ExtraTreesClassifier()
#     model.fit(x_train, y_train)

#     y_pred = model.predict(x_test)
#     print(classification_report(y_test, y_pred))
    
#     count_zeros_new = np.sum(y_new == 0)
#     count_ones_new = np.sum(y_new == 1)

#     print(f"Count of 0s (No Heart Attack): {count_zeros_new}")
#     print(f"Count of 1s (Heart Attack): {count_ones_new}")

In [20]:
# def compute_knn(X, y, minority_class, k):
#     minority_instances = X[y == minority_class]
#     nbrs = NearestNeighbors(n_neighbors=k).fit(X)
#     distances, indices = nbrs.kneighbors(minority_instances)
#     return indices

# def identify_borderline_instances(X, y, indices, minority_class, k):
#     borderline_instances = []
#     for i, idx in enumerate(indices):
#         ksame = sum(y[idx] == minority_class)
#         kmajority = k - ksame
#         if ksame < k and kmajority > ksame:
#             borderline_instances.append(i)
#     return borderline_instances

# def generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples):
#     synthetic_samples = []
#     num_borderline = len(borderline_indices)

#     samples_per_instance = num_samples // num_borderline
#     for i in borderline_indices:
#         for _ in range(samples_per_instance):
#             rand_idx = np.random.choice(indices[i])
#             xj = X[rand_idx]
#             xi = X[i]
#             rand = np.random.rand()
#             xnew = xi + rand * (xj - xi)
#             synthetic_samples.append(xnew)

#     remaining_samples = num_samples - len(synthetic_samples)
#     for _ in range(remaining_samples):
#         i = np.random.choice(borderline_indices)
#         rand_idx = np.random.choice(indices[i])
#         xj = X[rand_idx]
#         xi = X[i]
#         rand = np.random.rand()
#         xnew = xi + rand * (xj - xi)
#         synthetic_samples.append(xnew)
#     return np.array(synthetic_samples)

# def smote(X, y, minority_class, k, num_samples):
#     indices = compute_knn(X, y, minority_class, k)
#     borderline_indices = identify_borderline_instances(X, y, indices, minority_class, k)
#     synthetic_samples = generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples)
    
#     X_new = np.vstack((X, synthetic_samples))
#     y_new = np.hstack((y, np.array([minority_class] * len(synthetic_samples))))
    
#     return X_new, y_new

# if __name__ == "__main__":
#     target = data['TenYearCHD']
#     columns_to_drop = ['TenYearCHD']
#     features = data.drop(columns_to_drop, axis=1)
#     X = features.values
#     y = target.values

#     count_zeros_original = np.sum(y == 0)
#     count_ones_original = np.sum(y == 1)
#     print(f"0s (No Heart Attack): {count_zeros_original}")
#     print(f"1s (Heart Attack): {count_ones_original}")
    
#     x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     print(f"Training size: {x_train.shape}, Test size: {x_test.shape}")
    
#     minority_class =1
#     k =5
#     num_samples = 2000
#     x_train_new, y_train_new = smote(x_train, y_train, minority_class, k, num_samples)
#     print(f"New training size: {x_train_new.shape}, Original training size: {x_train.shape}")

#     model = ExtraTreesClassifier(class_weight='balanced')
#     model.fit(x_train_new, y_train_new)
#     y_pred = model.predict(x_test)
#     print(classification_report(y_test, y_pred))

#     count_zeros_new = np.sum(y_train_new == 0)
#     count_ones_new = np.sum(y_train_new == 1)

#     print(f"0s (No Heart Attack) in new training: {count_zeros_new}")
#     print(f"1s (Heart Attack) in new training: {count_ones_new}")

# ADASYN

In [21]:
# from imblearn.over_sampling import ADASYN

# def compute_knn(X, y, minority_class, k):
#     minority_instances = X[y == minority_class]
#     nbrs = NearestNeighbors(n_neighbors=k).fit(X)
#     distances, indices = nbrs.kneighbors(minority_instances)
#     return indices

# def identify_borderline_instances(X, y, indices, minority_class, k):
#     borderline_instances = []
#     for i, idx in enumerate(indices):
#         ksame = sum(y[idx] == minority_class)
#         kmajority = k - ksame
#         if ksame < k and kmajority > ksame:
#             borderline_instances.append(i)
#     return borderline_instances

# def generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples):
#     synthetic_samples = []
#     num_borderline = len(borderline_indices)
#     samples_per_instance = num_samples // num_borderline
#     for i in borderline_indices:
#         for _ in range(samples_per_instance):
#             rand_idx = np.random.choice(indices[i])
#             xj = X[rand_idx]
#             xi = X[i]
#             # Interpolation
#             rand = np.random.rand()
#             xnew = xi + rand * (xi - xj)
#             synthetic_samples.append(xnew)
#     remaining_samples = num_samples - len(synthetic_samples)
#     for _ in range(remaining_samples):
#         i = np.random.choice(borderline_indices)
#         rand_idx = np.random.choice(indices[i])
#         xj = X[rand_idx]
#         xi = X[i]
#         rand = np.random.rand()
#         xnew = xi + rand * (xi - xj)
#         synthetic_samples.append(xnew)
#     return np.array(synthetic_samples)


# def smote(X, y, minority_class, k, num_samples):
#     indices = compute_knn(X, y, minority_class, k)
#     borderline_indices = identify_borderline_instances(X, y, indices, minority_class, k)
#     synthetic_samples = generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples)
    
#     X_new = np.vstack((X, synthetic_samples))
#     y_new = np.hstack((y, np.array([minority_class] * len(synthetic_samples))))
    
#     return X_new, y_new

# if __name__ == "__main__":
    
#     target = data['TenYearCHD']
#     columns_to_drop = ['TenYearCHD']
#     features = data.drop(columns_to_drop, axis=1)

#     X = features.values
#     y = target.values

#     count_zeros_original = np.sum(y == 0)
#     count_ones_original = np.sum(y == 1)

#     print(f"Original Count of 0s (No Heart Attack): {count_zeros_original}")
#     print(f"Original Count of 1s (Heart Attack): {count_ones_original}")

#     ada = ADASYN(sampling_strategy='minority', random_state=42)
#     X_new, y_new = ada.fit_resample(X, y)

#     x_train, x_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

#     model = RandomForestClassifier(random_state=42)
#     model.fit(x_train, y_train)
#     y_pred = model.predict(x_test)
#     print(classification_report(y_test, y_pred))

#     count_zeros_new = np.sum(y_new == 0)
#     count_ones_new = np.sum(y_new == 1)

#     print(f"Count of 0s (No Heart Attack): {count_zeros_new}")
#     print(f"Count of 1s (Heart Attack): {count_ones_new}")

# Combining Oversampling and Undersampling

In [135]:
data= pd.read_csv('/kaggle/input/framingham/framingham.csv')
data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})
data['currentSmoker']= data['currentSmoker'].map({'Yes':1, 'No':0})
data['diabetes']= data['diabetes'].map({'Yes':1, 'No':0})
data['cigsPerDay'] = data['cigsPerDay'].fillna(data['cigsPerDay'].mean())
data['BPMeds'] = data['BPMeds'].fillna(data['BPMeds'].median())
data['totChol'] = data['totChol'].fillna(data['totChol'].mean())
data['BMI'] = data['BMI'].fillna(data['BMI'].median())
data['heartRate'] = data['heartRate'].fillna(data['heartRate'].median())
data['glucose'] = data['glucose'].fillna(data['glucose'].median())
data['education'] = data['education'].fillna(data['education'].mode()[0])
# data
def remove_outliers_iqr(df, column):
    Q1= df[column].quantile(0.25)  
    Q3= df[column].quantile(0.75) 
    IQR= Q3- Q1                   
    lower_bound=Q1- 1.5* IQR    
    upper_bound= Q3+ 1.5* IQR     

    return df[(df[column]>= lower_bound) & (df[column]<= upper_bound)]

columns_with_outliers= ['cigsPerDay', 'BPMeds', 'totChol', 'BMI', 'heartRate', 'glucose']
for col in columns_with_outliers:
    data = remove_outliers_iqr(data, col)

print(data.shape)

(3655, 16)


In [144]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

if __name__ == "__main__":   
    target = data['TenYearCHD']
    columns_to_drop = ['TenYearCHD']
    features = data.drop(columns_to_drop, axis=1)

    X = features.values
    y = target.values
    
    count_zeros_original = np.sum(y == 0)
    count_ones_original = np.sum(y == 1)

    print(f"Original Count of 0s (No Heart Attack): {count_zeros_original}")
    print(f"Original Count of 1s (Heart Attack): {count_ones_original}")

    majority_count = count_zeros_original  
    minority_count = count_ones_original   

    desired_minority_count = majority_count
    num_samples_to_oversample = desired_minority_count - minority_count-100

    print(f"Desired Count of 1s after oversampling: {desired_minority_count}")
    print(f"Number of 1s to oversample: {num_samples_to_oversample}")

    over = ADASYN(sampling_strategy={1: desired_minority_count}, random_state=63)  
    under = RandomUnderSampler(sampling_strategy={0: majority_count}, random_state=63)  

    pipeline = Pipeline(steps=[('over', over), ('under', under)])

    X_resampled, y_resampled = pipeline.fit_resample(X, y)
    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(x_train, y_train)

    y_pred_proba = model.predict_proba(x_test)[:, 1]
    threshold = 0.5
    y_pred_adjusted = (y_pred_proba >= threshold).astype(int)

    print(classification_report(y_test, y_pred_adjusted))

Original Count of 0s (No Heart Attack): 3156
Original Count of 1s (Heart Attack): 499
Desired Count of 1s after oversampling: 3156
Number of 1s to oversample: 2557
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       644
           1       0.94      0.90      0.92       636

    accuracy                           0.92      1280
   macro avg       0.92      0.92      0.92      1280
weighted avg       0.92      0.92      0.92      1280



In [145]:
model = ExtraTreesClassifier(random_state=42, class_weight={0: 1, 1: 7})
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95       644
           1       0.97      0.94      0.95       636

    accuracy                           0.95      1280
   macro avg       0.95      0.95      0.95      1280
weighted avg       0.95      0.95      0.95      1280



In [146]:
train_preds = model.predict(x_train)
train_accuracy = accuracy_score(y_train, train_preds)
print("Training Accuracy: {:.2f}%".format(train_accuracy * 100))
test_preds = model.predict(x_test)
test_accuracy = accuracy_score(y_test, test_preds)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

Training Accuracy: 100.00%
Test Accuracy: 95.31%


In [147]:
classifiers = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(n_estimators=1500, random_state=42, class_weight={0: 1, 1: 5}),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighbors": KNeighborsClassifier(),
    #"RadiusNeighbors": RadiusNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "MLPClassifier": MLPClassifier(random_state=42)
}
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall', 'Specificity', 'ROC-AUC', 'Confusion Matrix'])

def evaluate_classifiers(classifiers, x_train, x_test, y_train_resampled, y_test):
    global results_df
    for name, clf in classifiers.items():
        try:
            clf.fit(x_train, y_train) 
            y_pred = clf.predict(x_test)  

            cm = confusion_matrix(y_test, y_pred)
            if len(cm.ravel()) == 4:  
                tn, fp, fn, tp = cm.ravel()
                specificity = tn / (tn + fp) 
            else:
                specificity = None

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred, output_dict=True)
            precision = report['weighted avg']['precision']
            recall = report['weighted avg']['recall']

            if hasattr(clf, "predict_proba"): 
                y_proba = clf.predict_proba(x_test)[:, 1]
                roc_auc = roc_auc_score(y_test, y_proba)
            else:
                roc_auc = None
                
            new_row = pd.DataFrame({
                'Model': [name],
                'Accuracy': [acc],
                'F1 Score': [f1],
                'Precision': [precision],
                'Recall': [recall],
                'Specificity': [specificity],
                'ROC-AUC': [roc_auc],
                'Confusion Matrix': [cm]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        except (ValueError, NotFittedError) as e:
            print(f"Error with classifier {name}: {e}")
            
evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
print(results_df)

/tmp/ipykernel_36/2139533368.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Info] Number of positive: 2604, number of negative: 2512
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001834 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3084
[LightGBM] [Info] Number of data points in the train set: 5116, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.508991 -> initscore=0.035969
[LightGBM] [Info] Start training from score 0.035969
                   Model  Accuracy  F1 Score  Precision    Recall  \
0           RandomForest  0.917969  0.917901   0.919057  0.917969   
1               AdaBoost  0.861719  0.861426   0.864338  0.861719   
2             ExtraTrees  0.953906  0.953900   0.954035  0.953906   
3       GradientBoosting  0.885938  0.885120   0.896308  0.885938   
4                Bagging  0.907813  0.907672   0.909891  0.907813   
5   HistGradientBoosting  0.908594  0.908312   0.913067  0.908594   
6           DecisionTree  0.

In [ ]:
# results_df.to_csv('model_evaluation_results.csv', index=False)

In [142]:
from keras.callbacks import EarlyStopping

def compute_knn(X, y, minority_class, k):
    minority_instances = X[y == minority_class]
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)
    distances, indices = nbrs.kneighbors(minority_instances)
    return indices

def identify_borderline_instances(X, y, indices, minority_class, k):
    borderline_instances = []
    for i, idx in enumerate(indices):
        ksame = sum(y[idx] == minority_class)
        kmajority = k - ksame
        if ksame < k and kmajority > ksame:
            borderline_instances.append(i)
    return borderline_instances

def generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples):
    synthetic_samples = []
    num_borderline = len(borderline_indices)
    samples_per_instance = num_samples // num_borderline
    for i in borderline_indices:
        for _ in range(samples_per_instance):
            rand_idx = np.random.choice(indices[i])
            xj = X[rand_idx]
            xi = X[i]
            rand = np.random.rand()
            xnew = xi + rand * (xj - xi)
            synthetic_samples.append(xnew)
    remaining_samples = num_samples - len(synthetic_samples)
    for _ in range(remaining_samples):
        i = np.random.choice(borderline_indices)
        rand_idx = np.random.choice(indices[i])
        xj = X[rand_idx]
        xi = X[i]
        rand = np.random.rand()
        xnew = xi + rand * (xj - xi)
        synthetic_samples.append(xnew)
    return np.array(synthetic_samples)

def smote(X, y, minority_class, k, num_samples):
    indices = compute_knn(X, y, minority_class, k)
    borderline_indices = identify_borderline_instances(X, y, indices, minority_class, k)
    synthetic_samples = generate_synthetic_samples(X, y, borderline_indices, indices, minority_class, num_samples)
    X_new = np.vstack((X, synthetic_samples))
    y_new = np.hstack((y, np.array([minority_class] * len(synthetic_samples))))
    return X_new, y_new

def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation='relu')(input_layer)
    decoder = Dense(input_dim, activation='linear')(encoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')
    return autoencoder

if __name__ == "__main__":
    target = data['TenYearCHD']
    columns_to_drop = ['TenYearCHD']
    features = data.drop(columns_to_drop, axis=1)
    X = features.values
    y = target.values

    count_zeros_original = np.sum(y == 0)
    count_ones_original = np.sum(y == 1)
    print(f"Original Count of 0s (No Heart Attack): {count_zeros_original}")
    print(f"Original Count of 1s (Heart Attack): {count_ones_original}")

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    input_dim = X_scaled.shape[1]
    encoding_dim = input_dim // 2  
    
    autoencoder = build_autoencoder(input_dim, encoding_dim)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    autoencoder.fit(X_scaled, X_scaled, epochs=200, batch_size=16, validation_split=0.2, verbose=2, callbacks=[early_stopping])
    
    X_imputed = autoencoder.predict(X_scaled)
    X_imputed = scaler.inverse_transform(X_imputed)

    X_final = np.where(np.isnan(X), X_imputed, X)

    over = FunctionSampler(func=ADASYN(sampling_strategy='minority', random_state=42).fit_resample)
    under = FunctionSampler(func=RandomUnderSampler(sampling_strategy='majority', random_state=42).fit_resample)

    pipeline = Pipeline(steps=[('over', over), ('under', under)])
    
    X_resampled, y_resampled = pipeline.fit_resample(X_final, y)
    k = 5 
    num_samples = 333 
    minority_class = 1  

    X_final_resampled, y_final_resampled = smote(X_resampled, y_resampled, minority_class, k, num_samples)
    x_train, x_test, y_train, y_test = train_test_split(X_final_resampled, y_final_resampled, test_size=0.2, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(classification_report(y_test, y_pred))

    count_zeros_new = np.sum(y_final_resampled == 0)
    count_ones_new = np.sum(y_final_resampled == 1)
    print(f"Final Count of 0s (No Heart Attack) in new training set: {count_zeros_new}")
    print(f"Final Count of 1s (Heart Attack) in new training set: {count_ones_new}")

Original Count of 0s (No Heart Attack): 3156
Original Count of 1s (Heart Attack): 499
Epoch 1/200
183/183 - 1s - 7ms/step - loss: 1.1249 - val_loss: 0.9809
Epoch 2/200
183/183 - 0s - 2ms/step - loss: 0.8345 - val_loss: 0.7939
Epoch 3/200
183/183 - 0s - 2ms/step - loss: 0.6911 - val_loss: 0.6706
Epoch 4/200
183/183 - 0s - 2ms/step - loss: 0.5912 - val_loss: 0.5799
Epoch 5/200
183/183 - 0s - 2ms/step - loss: 0.5194 - val_loss: 0.5143
Epoch 6/200
183/183 - 0s - 2ms/step - loss: 0.4680 - val_loss: 0.4675
Epoch 7/200
183/183 - 0s - 2ms/step - loss: 0.4298 - val_loss: 0.4332
Epoch 8/200
183/183 - 0s - 2ms/step - loss: 0.4012 - val_loss: 0.4059
Epoch 9/200
183/183 - 0s - 2ms/step - loss: 0.3786 - val_loss: 0.3870
Epoch 10/200
183/183 - 0s - 2ms/step - loss: 0.3608 - val_loss: 0.3708
Epoch 11/200
183/183 - 0s - 2ms/step - loss: 0.3465 - val_loss: 0.3582
Epoch 12/200
183/183 - 0s - 2ms/step - loss: 0.3341 - val_loss: 0.3464
Epoch 13/200
183/183 - 0s - 2ms/step - loss: 0.3238 - val_loss: 0.3372


In [143]:
model =ExtraTreesClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       640
           1       0.95      0.94      0.94       689

    accuracy                           0.94      1329
   macro avg       0.94      0.94      0.94      1329
weighted avg       0.94      0.94      0.94      1329



In [ ]:
# gboost_model = GradientBoostingClassifier(random_state=42)
# gboost_model.fit(x_train, y_train)

# y_gboost_pred = gboost_model.predict(x_test)
# print(classification_report(y_test, y_gboost_pred))

In [ ]:
# xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
# xgb_model.fit(x_train, y_train)

# y_xgb_pred = xgb_model.predict(x_test)
# print(classification_report(y_test, y_xgb_pred))

In [ ]:
# import statsmodels.api as sm

# imp = sm.MICEData(data)
# data = imp.data
# data



# target = data['TenYearCHD']
# columns_to_drop = ['TenYearCHD']
# features = data.drop(columns_to_drop, axis=1)
# X = features.values
# y = target.values
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# base_learners = [
#     ('dt', DecisionTreeClassifier()), 
#     ('knn', KNeighborsClassifier()),
#     ('et', ExtraTreesClassifier()),  # Extra Trees Classifier
#     ('rf', RandomForestClassifier()),  # Random Forest Classifier
#     ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))  # XGBoost
# ]

# meta_model = RandomForestClassifier()
# stack_model = StackingClassifier(estimators=base_learners, final_estimator=meta_model)
# stack_model.fit(x_train, y_train)

# y_pred = stack_model.predict(x_test)
# accuracy = accuracy_score(y_test, y_pred)

# print(f"Accuracy: {accuracy:.2f}")